In [14]:
import pandas as pd
from dateutil import parser
node_df = pd.read_csv("../../data/node_df.csv")


## 2. Семантический вектор (простая bag-of-words или TF-IDF).

**Что именно делаем**:

1. Формируем текстовое описание ноды — это конкатенация:
    - screen
    - feature
    - action
2. Берём одно значение для каждой node_id (все события ноды — одинаковые).
3. Генерируем TF-IDF вектора.
4. Присоединяем их к node_features.

In [15]:
# semantic_embeddings.py
# TF-IDF embedding для screen/feature/action каждой ноды

import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

def build_node_text_embeddings(df: pd.DataFrame, max_features: int = 64):
    """
    На вход: события (event-level) df.
    На выход: DataFrame с колонкой 'text_embedding' (np.array(dtype=float)),
              index=node_id.
    
    Требуемые колонки:
    'node_id', 'screen', 'feature', 'action'
    """

    required = {'node_id', 'screen', 'feature', 'action'}
    missing = required - set(df.columns)
    if missing:
        raise ValueError(f"Missing columns: {missing}")

    # --- 1. Формируем текст для каждой строки ---
    df = df.copy()
    df['node_text'] = (
        df['screen'].astype(str) + " " +
        df['feature'].astype(str) + " " +
        df['action'].astype(str)
    )

    # --- 2. Выбираем одно представление на ноду ---
    node_text = (
        df.groupby('node_id')['node_text']
        .agg(lambda s: s.iloc[0])  # достаточно первой строки
    )

    # --- 3. TF-IDF ---
    vectorizer = TfidfVectorizer(
        max_features=max_features,
        stop_words=None,
        lowercase=True
    )
    X = vectorizer.fit_transform(node_text.tolist())

    # --- 4. Конверт в numpy и DataFrame ---
    emb = pd.DataFrame(
        list(X.toarray()),
        index=node_text.index,
        columns=[f"tfidf_{i}" for i in range(X.shape[1])]
    )

    # Можно оставить как вектора
    emb['text_embedding'] = emb.apply(lambda row: row.values.astype(float), axis=1)

    return emb[['text_embedding']]


In [16]:
df = node_df.copy()

# после загрузки исходных событий
text_emb = build_node_text_embeddings(df)

# Объединение
node_df = pd.merge(node_df, text_emb, on='node_id', how='left')
# Сохранение в Parquet (сохраняет типы данных!)
node_df.to_parquet("../../data/node_semantic_df.parquet", index=True)

# Чтение
# df_loaded = pd.read_parquet("../../data/node_semantic_df.parquet")

node_df.head()


,node_id,screen,feature,action,total_visits,sessions_with_node,avg_visits_per_session,median_visits_per_session,avg_session_length,avg_time_on_page_seconds,...,churn_count,churn_rate,bounce_count,bounce_rate,avg_age,male_ratio,device_vendor_top,device_type_top,os_top,text_embedding
0,0072f89b60d46ef6f2094949d8831f13,Важное,Просмотр уведомления,Тап на уведомление,7489,4609,1.624864,1.0,4.175092,30.737356,...,389,0.051943,2977,0.397516,46.105622,0.464548,Apple,phone,Android,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,02b207cc24a78c1942161bafc72fe532,Еще,Переход в раздел 'Опросы и собрания собственни...,Тап на кнопку 'Опросы и собрания собственников',4167,3658,1.139147,1.0,4.681520,63.280683,...,246,0.059035,2291,0.549796,46.174706,0.438445,Apple,phone,Android,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,03fa5765e1bee3c6ba8b9a139635c46a,Новый адрес,Активация гостевого доступа,Тап на кнопку 'Активировать',9,5,1.800000,1.0,4.400000,7.000000,...,0,0.000000,3,0.333333,49.444444,0.777778,Sony,phone,Android,"[0.0, 0.5459835934470357, 0.0, 0.0, 0.0, 0.0, ..."
3,05aa62cfe2beb31d4ecc652cddec5689,Объявления,Редактирование опубликованного объявления,Тап на кнопку 'Редактировать',3,3,1.000000,1.0,7.000000,62.000000,...,0,0.000000,1,0.333333,48.666667,0.333333,Apple,phone,iOS,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,061da77ad7d449a342174810fbf72350,Гостевой доступ,Раскрытие вкладки 'Архив',Тап на кнопку 'Архив',6,6,1.000000,1.0,16.333333,2.500000,...,0,0.000000,2,0.333333,32.333333,0.500000,Samsung,phone,Android,"[0.0, 0.0, 0.0, 0.0, 0.5293365357504765, 0.0, ..."
